DSML investigation:

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part five

### Due 29.10 (You get an extra hour!)

By now you should have 19 suspects left.
More information came in that suggests that the rogue agent is tampering with the sentiment annotation system of the SIMF which analyses news documents and marks their sentiment of intelligence analysis tasks.

This annotation is crutial to identify documents expressing negativity towards Switzerland and its allies.

Each document contains a column which shows which user accessed it. We know that the rogue agent accessed only the documents whose probability of negative sentiment was between 0.4 and 0.5 initially and beween 0.1 and 0.2 according to the SIMF classifier which you can load onto your model.


[You can find more models on this link](https://huggingface.co/models?sort=trending)


In [1]:
%%capture
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs
!pip install transformers[torch]
!pip install accelerate -U
# Import required packages
from transformers import pipeline, DataCollatorWithPadding
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

torch.cuda.is_available()


2023-11-13 08:56:33.991623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# 1. Getting to know our data

In [2]:
df = pd.read_excel('https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%205/data/Reduced_Set_2100.xlsx')

In [3]:
df.head(2)

,company,title,news,evaluation,year,month,day
0,APPLE,Tourists snap up British iPads to smuggle into...,IT'S the digital version of the slow boat to C...,negative,2011,4,17
1,CHEVRON,AFTER SEATTLE; Anarchists get organized.,"For Juliette Beck, it began with the story of ...",negative,2000,4,17


# 2. Re-evaluating with SIMF's model:
Evaluate the sentiment on the title column using a sentiment pipeline trained on the `finiteautomata/bertweet-base-sentiment-analysis`model


_This may take a while_

In [4]:
# Your code here
sentiment_pipeline = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

In [5]:
# Your code here
df['pred1'] = sentiment_pipeline(df['title'].tolist())
df

,company,title,news,evaluation,year,month,day,pred1
0,APPLE,Tourists snap up British iPads to smuggle into...,IT'S the digital version of the slow boat to C...,negative,2011,4,17,"{'label': 'NEU', 'score': 0.8835569620132446}"
1,CHEVRON,AFTER SEATTLE; Anarchists get organized.,"For Juliette Beck, it began with the story of ...",negative,2000,4,17,"{'label': 'NEU', 'score': 0.9666778445243835}"
2,Exxon Mobil,$10bn oil payout voided,SAN FRANCISCO: An appeal court yesterday voide...,negative,2001,11,9,"{'label': 'NEG', 'score': 0.8902711272239685}"
3,WAL MART STORES,Craft capitalism: Just do it yourself; Web mar...,The declaration from the Handmade Consortium m...,negative,2007,12,15,"{'label': 'POS', 'score': 0.6151897311210632}"
4,Exxon Mobil,Chevron gas project gets state green light,SYDNEY: Chevron has received final environment...,negative,2007,9,8,"{'label': 'NEU', 'score': 0.5660675168037415}"
...,...,...,...,...,...,...,...,...
2095,CHEVRON,No twig left behind; Old into new. Wood waste ...,The future of Canada's forests may lie in appl...,positive,2008,4,22,"{'label': 'NEU', 'score': 0.74048912525177}"
2096,CISCO SYSTEMS,Video Calls Replacing Face-to-Face Meetings,"Kim Woong-nam, who works for a mid-sized tradi...",positive,2008,5,26,"{'label': 'NEU', 'score': 0.9394232630729675}"
2097,WAL MART STORES,Unilever goes green in a small way,Sluggish sales in the $500 million laundry det...,positive,2008,3,3,"{'label': 'POS', 'score': 0.510532021522522}"
2098,GENERAL ELECTRIC,GE takes Leap 56 to CF34 successor INTRODUCTIO...,GENERAL ELECTRIC believes a new centreline suc...,positive,2008,5,13,"{'label': 'POS', 'score': 0.542725682258606}"


## 2.1 How many of the total entries match both the SIMF model **and** the hugginface model?

In [6]:
# Your code here
df['pred1'] = list(map(lambda x: x['label'].lower(), df['pred1'].to_list()))


In [7]:
df['evaluation'] = list(map(lambda x: x.lower()[0:3], df['evaluation'].to_list()))

In [8]:
df['match'] = np.where((df['evaluation'] == df['pred1']), df['pred1'], 0)

In [9]:
df_match = df[df['match']!=0]
df_match

,company,title,news,evaluation,year,month,day,pred1,match
2,Exxon Mobil,$10bn oil payout voided,SAN FRANCISCO: An appeal court yesterday voide...,neg,2001,11,9,neg,neg
12,MICROSOFT,"Greenpeace slams Microsoft, Nintendo's 'toxic'...",AMSTERDAM - Microsoft and Nintendo are taking ...,neg,2007,11,29,neg,neg
13,PEPSICO,PESTICIDE PANIC,NEW DELHI: India's top court has asked local s...,neg,2006,8,6,neg,neg
14,Exxon Mobil,"High-cost oil projects may be wasted, report says",More than $1tn (£600bn) is being gambled on hi...,neg,2014,5,8,neg,neg
23,Exxon Mobil,MORE TOXIC SLUDGE FEARED IN WATERWAY,WORKERS BUILDING a park on Newtown Creek hit u...,neg,2005,2,24,neg,neg
...,...,...,...,...,...,...,...,...,...
2087,COCA COLA,"You said it, earthlings: WE WANT CHANGE; EARTH...",One billion people in 83 countries. Three year...,pos,2009,3,29,pos,pos
2094,CITIGROUP,HOMETOWN CEO LIKES WHAT HE SEES IN LENNAR,Which Tampa CEO sees potential gold in them th...,pos,2008,2,5,pos,pos
2097,WAL MART STORES,Unilever goes green in a small way,Sluggish sales in the $500 million laundry det...,pos,2008,3,3,pos,pos
2098,GENERAL ELECTRIC,GE takes Leap 56 to CF34 successor INTRODUCTIO...,GENERAL ELECTRIC believes a new centreline suc...,pos,2008,5,13,pos,pos


In [10]:
913/2100

0.43476190476190474

## 2.2 We will now focus on the entries that do not match
#### Identify all non matching entries

In [11]:
# Your code here
df_non_match = df[df['match']==0]
display(df_non_match)

,company,title,news,evaluation,year,month,day,pred1,match
0,APPLE,Tourists snap up British iPads to smuggle into...,IT'S the digital version of the slow boat to C...,neg,2011,4,17,neu,0
1,CHEVRON,AFTER SEATTLE; Anarchists get organized.,"For Juliette Beck, it began with the story of ...",neg,2000,4,17,neu,0
3,WAL MART STORES,Craft capitalism: Just do it yourself; Web mar...,The declaration from the Handmade Consortium m...,neg,2007,12,15,pos,0
4,Exxon Mobil,Chevron gas project gets state green light,SYDNEY: Chevron has received final environment...,neg,2007,9,8,neu,0
5,CHEVRON,RIVALS WILL ACCUSE BP,[…] BP was expected to be hung out to dry by r...,neg,2010,6,15,neu,0
...,...,...,...,...,...,...,...,...,...
2091,PEPSICO,At Deadline,"[…] WM, PEPSICO launch kiosk recycling program...",pos,2010,4,26,neu,0
2092,PFIZER,TAKING UP THE ISSUE OF RENEWABLE ENERGY,WE HAVE come to an age where we must seriously...,pos,2003,11,29,neu,0
2093,GENERAL ELECTRIC,Virgin Atlantic gives $8bn order to Boeing in ...,Virgin Atlantic dealt a massive blow to Airbus...,pos,2007,4,25,neu,0
2095,CHEVRON,No twig left behind; Old into new. Wood waste ...,The future of Canada's forests may lie in appl...,pos,2008,4,22,neu,0


## 2.3 How many of those entries were changed from negative to neutral or positive ?

In [12]:
# Your code here
df_change = df_non_match[df_non_match['pred1']=='neg']
df_change

,company,title,news,evaluation,year,month,day,pred1,match
702,BANK OF AMERICA,Letters: Anger simmers over energy bills; Thes...,"[…] Ending our addiction to fossil fuels, slas...",neu,2013,10,17,neg,0
715,VERIZON COMMUNICATIONS,So How Contaminated Is the Old Nuclear Plant?,"[…] From 1952 to 1966, when operations ceased ...",neu,2002,1,13,neg,0
722,CITIGROUP,Front: Carbon bubble 'creates global economic ...,The world could be heading for a major economi...,neu,2013,4,19,neg,0
733,CITIGROUP,Report says officials are shortsighted big spe...,Government officials are guilty of wasting mon...,neu,2003,11,27,neg,0
738,CITIGROUP,Miners to bear cost of climate toll,"RESOURCE companies, along with transporter Tol...",neu,2006,12,11,neg,0
...,...,...,...,...,...,...,...,...,...
2032,JOHNSON & JOHNSON,Energy: Internet hoax raises pressure over emi...,Environmentalists opened up a new front for cl...,pos,2007,12,4,neg,0
2034,MERCK & COMPANY,TRENDS RAIN-FOREST CHIC Maybe Ben & Jerry'...,[…] Roddick told the Mexican conclave of big-b...,pos,1995,9,29,neg,0
2037,WAL MART STORES,Vancouver snub 'disappoints' Wal-Mart,VANCOUVER - A day after Vancouver City Council...,pos,2005,6,30,neg,0
2053,GENERAL ELECTRIC,Climate group seeks legal action over hoax web...,"The US Climate Action Partnership, a broad coa...",pos,2007,12,6,neg,0


# 3. Use the ChangeLog dataframe to identify the usersID's who edited the tampered entries, and only the altered entries

In [13]:
changelog = pd.read_csv('https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%205/data/ChangeLog.csv')
display(changelog)

,UserID,title
0,[327047],Tourists snap up British iPads to smuggle into...
1,[401818],Tourists snap up British iPads to smuggle into...
2,[564061],Tourists snap up British iPads to smuggle into...
3,[446376],Tourists snap up British iPads to smuggle into...
4,[242912],AFTER SEATTLE; Anarchists get organized.
...,...,...
4164,[340001],GE takes Leap 56 to CF34 successor INTRODUCTIO...
4165,[327000],Award for bus captain who drives with a smile;...
4166,[820116],Award for bus captain who drives with a smile;...
4167,[790344],Award for bus captain who drives with a smile;...


## 3.1 Identifying the users who have edited tampered documents

In [14]:
# Your code here
changed = pd.merge(changelog, df_change, on="title")
listchanged = changed['UserID'].tolist()
print(listchanged)

['[140991]', '[628864]', '[711683]', '[398434]', '[241404]', '[535048]', '[780084]', '[679196]', '[850103]', '[515957]', '[581969]', '[601614]', '[792062]', '[910232]', '[628854]', '[741709]', '[638911]', '[265615]', '[888670]', '[720728]', '[128570]', '[476861]', '[151092]', '[191622]', '[123006]', '[897796]', '[223968]', '[825733]', '[682880]', '[359736]', '[717780]', '[781819]', '[631142]', '[404377]', '[551506]', '[425350]', '[297157]', '[942442]', '[911014]', '[789726]', '[299485]', '[269302]', '[136838]', '[872993]', '[793674]', '[640117]', '[808893]', '[628233]', '[353596]', '[585467]', '[649162]', '[544861]', '[345709]', '[837811]', '[280659]', '[888670]', '[517645]', '[249197]', '[945010]', '[823355]', '[651597]', '[490705]', '[131191]', '[268218]', '[539227]', '[679196]', '[235799]', '[610905]', '[405604]', '[543267]', '[413250]', '[446376]', '[949985]', '[769520]', '[932606]', '[949262]', '[284274]', '[774730]', '[666304]', '[317366]', '[743377]', '[583984]', '[833402]', '[3

## 3.2 Identifying the users who have edited non-tampered documents

In [15]:
# Your code here
unchanged = pd.merge(changelog, df_match, on="title")
listunchanged =unchanged['UserID'].tolist()
print(listunchanged)

['[654139]', '[162287]', '[682811]', '[412064]', '[767527]', '[762279]', '[446376]', '[623888]', '[792062]', '[932281]', '[105236]', '[764821]', '[464891]', '[505876]', '[789469]', '[372166]', '[387676]', '[618568]', '[719655]', '[108420]', '[334769]', '[99489]', '[688784]', '[228819]', '[224422]', '[820116]', '[345709]', '[165762]', '[154013]', '[296491]', '[509130]', '[708256]', '[851347]', '[680273]', '[133154]', '[579515]', '[772645]', '[372166]', '[306495]', '[750955]', '[549118]', '[466702]', '[504629]', '[247617]', '[205775]', '[245757]', '[133154]', '[222875]', '[188061]', '[874746]', '[393146]', '[702840]', '[899768]', '[796491]', '[649162]', '[96840]', '[499672]', '[539529]', '[623440]', '[441742]', '[897796]', '[277417]', '[945010]', '[448665]', '[903438]', '[571277]', '[208948]', '[795805]', '[103928]', '[881695]', '[647912]', '[899768]', '[740078]', '[316652]', '[359736]', '[685615]', '[206644]', '[296491]', '[427874]', '[397601]', '[939524]', '[446160]', '[925217]', '[238

## 3.3 combining the results from `3.1` and `3.2` to identify users who only edited tampered documents.

In [16]:
# Your code here
suspects = list(set(listchanged) - set(listunchanged))
#suspects.sort()
#suspects = list(set(suspects))
display(suspects)

['[825733]',
 '[628854]',
 '[261521]',
 '[754702]',
 '[541833]',
 '[455352]',
 '[409012]',
 '[902629]',
 '[410319]',
 '[628233]',
 '[200865]',
 '[402663]',
 '[743377]',
 '[131191]',
 '[165305]',
 '[785994]',
 '[745000]',
 '[793674]',
 '[223968]',
 '[339524]',
 '[564884]',
 '[108215]',
 '[943792]',
 '[411464]',
 '[544861]',
 '[539227]',
 '[703326]']

In [21]:
print('[628854]' in suspects)
print('[785994]' in suspects)
print('[793674]' in suspects)
print('[410319]' in suspects)
print('[638911]' in suspects)

True
True
True
True
False


# 4. Identifying important informations on the altered documents.

In this section we will use the TF-IDF text representation model to identify other important information on the altered documents.

In [17]:
# Your code here
corpus = df['news'].tolist()

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Using default tokenizer in TfidfVectorizer, use the "english" stop words, and the ngram
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words="english")

# Learn the vocabulary dictionary and return document-term matrix
features = tfidf.fit_transform(corpus)

# Visualize result in dataframe
results = pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names_out(),
    ).drop('title', axis=1)
results


,00,000,000b,000billion,000kg,000km,000kwh,000mt,000mwh,000sqkm,...,zoning,zoo,zoologist,zorigt,zoé,zse,zucker,zuckerberg,zune,zurich
0,0.0,0.054857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2096,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2097,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2098,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Keep the entries related to tampered documents
word_frequency = results.iloc[df_change.index.tolist()]

# Identify the record that stands out the most on the altered documents (ou can use the sum of the tokenizers results)

word_frequency.sum().sort_values(ascending=False)

carbon       5.759377
said         5.589130
energy       5.010363
climate      4.983437
emissions    4.267752
               ...   
10bn         0.000000
10bp         0.000000
10c          0.000000
10ft         0.000000
zhou         0.000000
Length: 21715, dtype: float64

In [22]:
results

,00,000,000b,000billion,000kg,000km,000kwh,000mt,000mwh,000sqkm,...,zoning,zoo,zoologist,zorigt,zoé,zse,zucker,zuckerberg,zune,zurich
0,0.0,0.054857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2096,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2097,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2098,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# How many records contain the word that stands out the most?
# e.g. if the word that stood out the most was "mouton", how many of the altered records contain the word mouton.


# How about the second word that stands out the most

# How about the third ?


# How about the fourth ?
